In [1]:
from skimage import io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
import scipy
from scipy import stats

%matplotlib inline

path1 = '/home/gs/DataScientist/planet'
trainPath = '/train-tif'
testPath = '/test-tif'

VERBOSE_INTERVAL = 2500

NUM_BINS = 64
MAX_PIX_VAL = 65535


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# definitions

def getImageData (filePath):
    stats = []
    try:
        img = io.imread(filePath)
        r, g, b, nir = img[:, :, 0], img[:, :, 1], img[:, :, 2], img[:, :, 3]
        if not 0 in (nir+r):
            n = (nir-r) / (nir+r)
        else:
            n = np.zeros(256)
                
        hr, bins = np.histogram(r,NUM_BINS,[0, MAX_PIX_VAL])
        hg, bins = np.histogram(g,NUM_BINS,[0, MAX_PIX_VAL])
        hb, bins = np.histogram(b,NUM_BINS,[0, MAX_PIX_VAL])
        hnir, bins = np.histogram(nir,NUM_BINS,[0, MAX_PIX_VAL])
        ndvi, bins = np.histogram(n,NUM_BINS,[-1, +1])

        # stats
        stats.append( np.mean(r) )
        stats.append( np.mean(g) )
        stats.append( np.mean(b) )
        stats.append( np.mean(nir) )
        stats.append( np.std(r) )
        stats.append( np.std(g) )
        stats.append( np.std(b) )
        stats.append( np.std(nir) )
        stats.append( np.min(r) )
        stats.append( np.min(g) )
        stats.append( np.min(b) )
        stats.append( np.min(nir) )
        stats.append( np.max(r) )
        stats.append( np.max(g) )
        stats.append( np.max(b) )
        stats.append( np.max(nir) )

        stats.append( scipy.stats.kurtosis(r.ravel()) )
        stats.append( scipy.stats.kurtosis(g.ravel()) )
        stats.append( scipy.stats.kurtosis(b.ravel()) )
        stats.append( scipy.stats.kurtosis(nir.ravel()) )
        stats.append( scipy.stats.skew(r.ravel()) )
        stats.append( scipy.stats.skew(g.ravel()) )
        stats.append( scipy.stats.skew(b.ravel()) )
        stats.append( scipy.stats.skew(nir.ravel()) )
        
        # to array
        stats = np.array(stats)
        
    except Exception as  e:
        print ('{}  error reading file {}'.format(e, filePath))
        hr = np.zeros(NUM_BINS)
        hg = np.zeros(NUM_BINS)
        hb = np.zeros(NUM_BINS)
        hnir = np.zeros(NUM_BINS)
        ndvi = np.zeros(NUM_BINS)
        stats = np.zeros(24)

    r = stats
    return r

def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=2000):
    br = 0
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.01
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = "logloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20, verbose_eval = 50)
        br = model.best_iteration
        #print ('best iteration for DICT: {}'.format(br))
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds, verbose_eval = 50)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model, br



In [4]:
print (getImageData('/home/gs/DataScientist/planet/train-tif/train_1.tif'))

[  4.32636890e+03   3.66302528e+03   2.49403081e+03   7.11953702e+03
   7.98738145e+02   5.21727736e+02   9.17993052e+02   1.32179325e+03
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   5.73440000e+04   2.97420000e+04   6.14400000e+04   3.05750000e+04
   2.47096249e+03   5.22092194e+02   2.11534609e+03   8.36272554e+00
   4.42834118e+01   1.20072242e+01   3.97348957e+01   2.98028492e-01]


In [5]:
# read Y_train

print ('read train y...')

try:
    Y_train = pd.read_csv(path1+'/train.csv')
except:
    path1 = '/home/ec2-user/DataScientist/planet'
    Y_train = pd.read_csv(path1+'/train.csv')

print ('Y_train lines read: {}'.format(len(Y_train)))

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in Y_train['tags'].values])))
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}
#print(label_map)
#print
#print(inv_label_map)

Y_trainDict = {}
for i, row in Y_train.iterrows():
    name = row['image_name']
    tags = row['tags']
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    Y_trainDict[name] = targets

#print (Y_trainDict['train_0'])
#print (Y_trainDict['train_1'])
#print (Y_trainDict['train_2'])


# read train images

X_train = [] # arrays

Y_trainAll = []
X_train_id = []
lines = 0

print('Read train images')
path = os.path.join(path1+trainPath, '*.tif')
print (path)
files = glob.glob(path)
for fl in files:
    lines += 1
    if lines % VERBOSE_INTERVAL == 0:
        print ('  files read: {}'.format(lines))
    flbase = os.path.basename(fl)
    r = getImageData(fl)
    X_train.append(r)
    name = flbase.replace('.tif', '')
    X_train_id.append(name)
    Y_trainAll.append(Y_trainDict[name])


Y_trainAll = pd.DataFrame(Y_trainAll)

X_train = pd.DataFrame(X_train)
print ('X_train shape   : {}'.format(X_train.shape))
print ('Y_trainAll shape: {}'.format(Y_trainAll.shape))


X_test = [] # arrays
X_test_id = []
lines = 0 

print('Read test images')
path = os.path.join(path1+testPath, '*.tif')
print (path)
files = glob.glob(path)
for fl in files:
    lines += 1
    if lines % VERBOSE_INTERVAL == 0:
        print ('  files read: {}'.format(lines))
    flbase = os.path.basename(fl)
    r = getImageData(fl)
    X_test.append(r)
    name = flbase.replace('.tif', '')
    X_test_id.append(name)

X_test = pd.DataFrame(X_test)
print (X_test.shape)



read train y...
Y_train lines read: 40479
Read train images
/home/gs/DataScientist/planet/train-tif/*.tif
  files read: 2500
  files read: 5000


/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2546: UserWarning: unpack: string size must be a multiple of element size
  warnings.warn("unpack: %s" % e)


  files read: 7500
  files read: 10000
  files read: 12500
  files read: 15000
  files read: 17500
  files read: 20000
invalid offset  error reading file /home/gs/DataScientist/planet/train-tif/train_28173.tif
  files read: 22500
invalid offset  error reading file /home/gs/DataScientist/planet/train-tif/train_18772.tif
  files read: 25000
  files read: 27500
  files read: 30000
  files read: 32500
  files read: 35000
invalid TIFF file  error reading file /home/gs/DataScientist/planet/train-tif/train_5023.tif
  files read: 37500
  files read: 40000
X_train shape   : (40479, 24)
Y_trainAll shape: (40479, 17)
Read test images
/home/gs/DataScientist/planet/test-tif/*.tif
  files read: 2500
  files read: 5000
  files read: 7500
  files read: 10000
  files read: 12500
  files read: 15000
  files read: 17500
  files read: 20000
  files read: 22500
too many indices for array  error reading file /home/gs/DataScientist/planet/test-tif/test_17393.tif
  files read: 25000
  files read: 27500
  file

In [ ]:

# save data frames for later use

# Y_trainAll
# X_train
# X_test

import pickle
pickle.dump( Y_trainAll, open( "SAVE_yTrainAll.p", "wb" ) )
pickle.dump( X_train, open( "SAVE_xTrain.p", "wb" ) )
pickle.dump( X_test, open( "SAVE_xTest.p", "wb" ) )
pickle.dump( Y_trainDict, open( "SAVE_yTrainDict.p", "wb" ) )
pickle.dump( label_map, open( "SAVE_labelMap.p", "wb" ) )
pickle.dump( inv_label_map, open( "SAVE_invLabelMap.p", "wb" ) )



In [3]:
# load data
import pickle
Y_trainAll = pickle.load( open( "SAVE_yTrainAll.p", "rb" ) )
X_train = pickle.load( open( "SAVE_xTrain.p", "rb" ) )
X_test = pickle.load( open( "SAVE_xTest.p", "rb" ) )
Y_trainDict = pickle.load( open( "SAVE_yTrainDict.p", "rb" ) )
label_map = pickle.load( open( "SAVE_labelMap.p", "rb" ) )
inv_label_map = pickle.load( open( "SAVE_invLabelMap.p", "rb" ) )




In [6]:

# xgb cross validation

# save number of rounds!
brDict = {}

for i in range(0,17):
    print ('target: {} {}'.format(i, inv_label_map[i]))
    Y_train = Y_trainAll.ix[:,i]

    kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=2016)
    for dev_index, val_index in kf.split(range(X_train.shape[0])):
        dev_X, val_X = X_train.iloc[dev_index], X_train.iloc[val_index]
        dev_y, val_y = Y_train.iloc[dev_index], Y_train.iloc[val_index]
        preds, model, br = runXGB(dev_X, dev_y, val_X, val_y)
        brDict[i] = br

print ('bestRounds:')
print (brDict)

target: 0 slash_burn
[0]	train-logloss:0.683392	test-logloss:0.683414
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[50]	train-logloss:0.363079	test-logloss:0.364057
[100]	train-logloss:0.211032	test-logloss:0.213025
[150]	train-logloss:0.129741	test-logloss:0.132905
[200]	train-logloss:0.083727	test-logloss:0.08836
[250]	train-logloss:0.05655	test-logloss:0.063091
[300]	train-logloss:0.040126	test-logloss:0.048606
[350]	train-logloss:0.029733	test-logloss:0.040373
[400]	train-logloss:0.022984	test-logloss:0.035749
[450]	train-logloss:0.018311	test-logloss:0.033227
[500]	train-logloss:0.015131	test-logloss:0.031933
[550]	train-logloss:0.0129	test-logloss:0.031322
[600]	train-logloss:0.011268	test-logloss:0.031154
Stopping. Best iteration:
[613]	train-logloss:0.010905	test-logloss:0.031118

[0]	train-logloss:0.68344	test-logloss:0.683397
Multiple eval metrics have been passed: 'test-lo

In [7]:
# xgboost predict




predsDF = pd.DataFrame()
for i in range(0,17):
    print ('predicting feature ' + str(i))
    Y_train = Y_trainAll.ix[:,i]
    #print (Y_train.shape)
    preds, model, br = runXGB(X_train, Y_train, X_test, num_rounds=int(brDict[i]*1.33))
    predsDF[i] = preds

print (predsDF.shape)

predicting feature 0
predicting feature 1
predicting feature 2
predicting feature 3
predicting feature 4
predicting feature 5
predicting feature 6
predicting feature 7
predicting feature 8
predicting feature 9
predicting feature 10
predicting feature 11
predicting feature 12
predicting feature 13
predicting feature 14
predicting feature 15
predicting feature 16
(40669, 17)


In [ ]:
i = xgb.importance(model=model)
print (i)

In [ ]:
print (predsDF.head())
print (X_train_id[0:5])

In [15]:
# predsDF to prediction file

def mapf (arr):
    res = ''
    for i in range(0,17):
        if arr[i] > 0.2:
            res += inv_label_map[i] + ' '
    res = res.rstrip()
    return res

print(predsDF.shape)

textResults = []

for i, row in predsDF.iterrows():
    #print (i)
    #print (list(row))
    textResults.append ( mapf( list (row)))
    
print (textResults[0:5])

res = pd.DataFrame()
res['image_name'] = X_test_id
res['tags'] = textResults

print (res.head())

res.to_csv('SUB_24_0.01eta.csv', index=False)




(40669, 18)
['clear primary cultivation agriculture', 'clear primary', 'clear primary water', 'clear primary water agriculture', 'primary partly_cloudy']
   image_name                                   tags
0  test_34045  clear primary cultivation agriculture
1  test_32937                          clear primary
2  test_10537                    clear primary water
3  test_30945        clear primary water agriculture
4  test_13470                  primary partly_cloudy


In [19]:
df = predsDF
df['id'] = X_test_id
del df['name']
print(df.head())
df.to_csv(path1+'/RAW_24_0.01.csv', index=False)

          0         1         2         3         4         5         6  \
0  0.001932  0.975753  0.044092  0.999802  0.000131  0.000067  0.071908   
1  0.004689  0.941225  0.003031  0.999672  0.000246  0.000173  0.128573   
2  0.003650  0.968908  0.008507  0.999046  0.000363  0.000148  0.662868   
3  0.007619  0.951647  0.001586  0.993588  0.001233  0.000551  0.888229   
4  0.001607  0.026205  0.003400  0.998926  0.000355  0.000230  0.060977   

          7         8         9        10        11        12        13  \
0  0.000437  0.513123  0.010195  0.000030  0.010449  0.006295  0.002913   
1  0.003920  0.040732  0.041974  0.000376  0.001389  0.001515  0.001484   
2  0.005152  0.041653  0.010430  0.000094  0.025573  0.004803  0.001730   
3  0.015048  0.067510  0.024420  0.003458  0.062426  0.019092  0.000602   
4  0.043465  0.037631  0.966683  0.000154  0.007978  0.015514  0.001213   

         14        15        16          id  
0  0.681608  0.046154  0.025350  test_34045  
1  0.0